### 📌 리랭커(Reranker) 개요
- **RAG 파이프라인에서 검색된 결과를 재정렬(재랭킹)하는 모듈**
- 벡터 검색은 "의미적으로 비슷한 후보"를 뽑아줌  
- 리랭커는 후보들을 다시 **정밀하게 점수화 & 순위 조정**
- 방법:
  - **Cross-Encoder 기반** (쿼리 + 문서 함께 입력 → 정밀 점수 산출)
  - **외부 API 기반** (Cohere, Jina 등)
- 장점: 검색 정확도 ↑, 최종 답변 품질 향상


In [1]:
# # 01. 리랭커(Reranker) 개요
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import FAISS

# # 임베딩 + 벡터스토어 기본 검색
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# texts = ["고양이는 귀엽다", "강아지는 충성스럽다", "호랑이는 멋지다", "고양이는 집에서 많이 키운다"]
# vectorstore = FAISS.from_texts(texts, embeddings)

# query = "애완동물"
# docs = vectorstore.similarity_search(query, k=3)
# for d in docs:
#     print(d.page_content)

### 📌 CrossEncoderReranker (BGE-m3-reranker)
- **BAAI(bge)**에서 제공하는 Cross-Encoder 기반 리랭커
- 쿼리와 문서를 동시에 인코딩 → **정밀한 유사도 점수 산출**
- 장점:
  - 높은 정확도 (RAG 품질 크게 향상)
- 단점:
  - 느림 (모든 후보를 모델에 재입력해야 함)
- 주로 **후처리 단계**에서 top-k 후보 정렬에 사용


In [2]:
# # 02. CrossEncoderReranker (BGE-m3-reranker)
# from langchain_community.rerankers.cross_encoder import CrossEncoderReranker

# reranker = CrossEncoderReranker(model="BAAI/bge-reranker-large")

# query = "집에서 키우는 동물"
# results = reranker.rerank(query, docs)

# for r in results:
#     print(r.page_content, r.metadata.get("relevance_score"))

### 📌 CohereReranker
- **Cohere API** 기반 리랭커
- 미리 학습된 `rerank-english-v2.0` 등 제공
- 장점:
  - API 호출만으로 사용 (빠른 시작 가능)
  - 성능 안정적
- 단점:
  - 외부 API 비용 발생
- 클라우드 기반 RAG 서비스에 적합

In [3]:
# # 03. CohereReranker
# # pip install cohere
# from langchain_community.rerankers import CohereReranker

# reranker = CohereReranker(model="rerank-english-v2.0", api_key="YOUR_COHERE_API_KEY")

# query = "집에서 키우는 동물"
# results = reranker.rerank(query, docs)

# for r in results:
#     print(r.page_content, r.metadata.get("relevance_score"))

### 📌 JinaReranker
- **Jina AI**에서 제공하는 리랭커
- 오픈소스 & API 기반
- 장점:
  - 빠르고 가벼움
  - 영어 검색 성능 우수
- 단점:
  - 한국어 등 다국어 성능은 제한적


In [4]:
# # 04. JinaReranker
# # pip install jina-ai
# from langchain_community.rerankers import JinaReranker

# reranker = JinaReranker(model="jina-reranker-v1-base-en")

# query = "집에서 키우는 동물"
# results = reranker.rerank(query, docs)

# for r in results:
#     print(r.page_content, r.metadata.get("relevance_score"))

### 📌 FlashRankReranker
- **FlashRank**: 경량화된 리랭킹 모델
- 주로 **MS MARCO** 데이터셋 기반 소형 모델 사용
- 장점:
  - 빠른 속도 (실시간 검색에 적합)
  - 로컬 실행 가능 (API 의존 없음)
- 단점:
  - 대형 Cross-Encoder 대비 정밀도는 다소 낮음
- 적합한 경우:
  - **온프레미스 환경** (외부 API 불가)
  - **실시간 응답 속도 중시**

In [6]:
# # 05. FlashRankReranker
# # pip install flashrank
# from langchain_community.rerankers import FlashrankReranker

# reranker = FlashrankReranker(model="ms-marco-MiniLM-L-12-v2")

# query = "집에서 키우는 동물"
# results = reranker.rerank(query, docs)

# for r in results:
#     print(r.page_content, r.metadata.get("relevance_score"))

## ✅ 최종 정리

- **CrossEncoderReranker (BGE-m3)**  
  - 가장 정밀, 하지만 속도 느림 → 후처리 Top-k 정렬 적합

- **CohereReranker**  
  - 클라우드 API 기반, 안정적 성능 → SaaS/클라우드 서비스 적합

- **JinaReranker**  
  - 가볍고 빠름, 영어 검색 특화 → 영어 중심 서비스 적합

- **FlashRankReranker**  
  - 초경량, 로컬 실행 가능 → 온프레미스/실시간 서비스 적합

👉 벡터 검색(Recall) + 리랭커(Precision) 조합이 최종 RAG 성능을 크게 결정합니다.